# xử lý dữ lieu job description

Luu y: phai luu file vietnamese-stopwords.txt vao hdfs
Có thê lam voi ca tieng anh roi concatenate 
file stopword tieng anh: stop_words_english.txt

In [ ]:
# tạo list stopwords tiếng việt
from pyspark.sql import SparkSession

def get_stopwords_list(file_path):
    # Khởi tạo Spark Session
    spark = SparkSession.builder \
        .appName("ReadStopwords") \
        .getOrCreate()

    # 1. Đọc file text bằng spark context (trả về một RDD)
    # 2. .map(lambda x: x.strip()) để loại bỏ khoảng trắng thừa hoặc ký tự xuống dòng
    # 3. .collect() để chuyển dữ liệu từ các worker về máy chủ (driver) dưới dạng list
    stopwords_rdd = spark.sparkContext.textFile(file_path)
    stopwords_list = stopwords_rdd.map(lambda line: line.strip()).collect()
    
    # Lọc bỏ các dòng trống nếu có
    stopwords_list = [word for word in stopwords_list if word]
    
    spark.stop()
    return stopwords_list

# Sử dụng hàm
file_path = "path/to/your/vietnamese_stopwords.txt"
my_stopwords = get_stopwords_list(file_path)

print(f"Số lượng stopwords đã đọc: {len(my_stopwords)}")
print(my_stopwords[:10]) # In thử 10 từ đầu tiên

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml import Pipeline

# 1. Tách từ (Tokenization)
tokenizer = Tokenizer(inputCol="description", outputCol="words")

# 2. Loại bỏ từ dừng (Stopwords: "và", "của", "the", "is"...)
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
remover.setStopWords(my_stopwords)

# 3. Vector hóa (TF-IDF) - Giúp làm nổi bật các từ khóa quan trọng như "Python", "Big Data"
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# 4. Chuyển nhãn (Label) sang dạng số (ví dụ: Job Category)
label_stringIdx = StringIndexer(inputCol="category", outputCol="label")

In [ ]:
# recruitment_analysis.py
# ĐÂY LÀ CLASSIFICATION

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def main():
    # Khởi tạo Spark Session
    spark = SparkSession.builder \
        .appName("RecruitmentBigDataAnalysis") \
        .getOrCreate()

    # Load dữ liệu (CSV/Parquet từ HDFS hoặc S3)
    df = spark.read.parquet("hdfs:///data/recruitment_data.parquet")

    # Pipeline stages: Feature Engineering + Model
    # Giả sử đã định nghĩa các bước tiền xử lý ở trên
    lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

    pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, label_stringIdx, lr])

    # Chia dữ liệu Train/Test
    (trainingData, testData) = df.randomSplit([0.8, 0.2], seed=1234)

    # Huấn luyện mô hình
    model = pipeline.fit(trainingData)

    # Dự đoán
    predictions = model.transform(testData)

    # Đánh giá độ chính xác
    evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    print(f"Test Accuracy: {accuracy:.2%}")

    # Lưu mô hình để sử dụng lại
    model.save("hdfs:///models/job_classifier_model")

if __name__ == "__main__":
    main()

In [ ]:
# ĐÂY LÀ GỢI Ý RECOMMENDATION MODEL
# Dùng ALS (Alternating Least Squares) cho hệ thống gợi ý việc làm dựa trên hồ sơ ứng viên và các công việc đã ứng tuyển trước đó.


In [ ]:
# Sử dung mô hình sau khi đã train

from pyspark.ml import PipelineModel

# Tải mô hình đã lưu
loaded_model = PipelineModel.load("hdfs:///models/job_classifier_model")

# Sử dụng mô hình vừa tải để dự đoán
new_predictions = loaded_model.transform(fresh_data)

